In [10]:
import os
import sys
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from pathlib import Path

# --- Path Setup ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

# --- Imports from src ---
from config import CONFIG
from skeleton_defs import SKELETON_HIERARCHY  # The file we created
# (Optional: Import interpolation utils if you have them in src)

# --- Directories ---
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
DERIV_01 = os.path.join(PROJECT_ROOT, CONFIG['derivatives_dir'], "step_01_parse")
DERIV_02 = os.path.join(PROJECT_ROOT, CONFIG['derivatives_dir'], "step_02_preprocess")
QC_02 = os.path.join(PROJECT_ROOT, CONFIG['qc_dir'], "step_02_preprocess")

os.makedirs(DERIV_02, exist_ok=True)
os.makedirs(QC_02, exist_ok=True)

print("Ready. Output directory:", DERIV_02)

Ready. Output directory: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_02_preprocess


In [11]:
# Derive the parquet file from the CSV path in config
# This ensures notebook 02 always processes the same file as notebook 01
csv_filename = Path(CONFIG['current_csv']).stem  # Gets filename without extension
RUN_ID = csv_filename
PARQUET_PATH = Path(DERIV_01) / f"{RUN_ID}__parsed_run.parquet"

if not PARQUET_PATH.exists():
    print(f"❌ ERROR: Expected parquet file not found: {PARQUET_PATH}")
    print(f"Did you run notebook 01 first?")
    raise FileNotFoundError(f"Parquet file not found: {PARQUET_PATH}")

print(f"Loading Run ID: {RUN_ID}")
print(f"File: {PARQUET_PATH}")

df_raw = pd.read_parquet(PARQUET_PATH)
print(f"Shape: {df_raw.shape}")

Loading Run ID: 734_T1_P2_R1_Take 2025-12-01 02.28.24 PM
File: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_01_parse\734_T1_P2_R1_Take 2025-12-01 02.28.24 PM__parsed_run.parquet
Shape: (19617, 359)


In [12]:
# --- Cell 3: Filter Out Fingers/Toes & Print Remaining Joints ---

def filter_and_verify_joints(df, keywords_to_drop):
    """
    1. Removes columns containing specific keywords (Fingers, Toes).
    2. Prints a clean list of the remaining joints for verification.
    """
    print(f"\n{'='*20} JOINT FILTERING & VERIFICATION {'='*20}")
    
    # --- 1. Filter ---
    initial_cols = len(df.columns)
    cols_to_drop = []
    
    for col in df.columns:
        if any(kw in col for kw in keywords_to_drop):
            cols_to_drop.append(col)
            
    if cols_to_drop:
        df_clean = df.drop(columns=cols_to_drop)
        dropped_count = len(cols_to_drop)
        print(f"📉 Dropped {dropped_count} columns matching: {keywords_to_drop}")
    else:
        df_clean = df
        print("ℹ️ No columns matched the filter.")

    # --- 2. Extract Remaining Joint Names ---
    # We look at the prefix before '__' (e.g., 'Hips__px' -> 'Hips')
    remaining_joints = sorted(list(set([c.split('__')[0] for c in df_clean.columns if '__' in c])))
    
    # --- 3. Print Report ---
    print(f"\n📊 STATUS REPORT:")
    print(f"   Original Columns: {initial_cols}")
    print(f"   Current Columns:  {len(df_clean.columns)}")
    print(f"   Unique Joints Remaining: {len(remaining_joints)}")
    
    print(f"\n✅ LIST OF JOINTS KEPT ({len(remaining_joints)}):")
    print("-" * 40)
    
    # Print in nice columns or list
    # Printing 4 joints per row for readability
    for i in range(0, len(remaining_joints), 4):
        print(f"   {', '.join(remaining_joints[i:i+4])}")
        
    print("-" * 40)
    print(f"{'='*60}\n")
    
    return df_clean

# --- CONFIGURATION ---
# מילים שאם הן מופיעות בשם המפרק - הוא יימחק.
# שים לב: "Toe" מוחק גם את "ToeBase". אם אתה רוצה את ToeBase, תוריד את "Toe" מהרשימה.
DROP_KEYWORDS = ["Thumb", "Index", "Middle", "Ring", "Pinky"] 
# הוסף את "Toe" לרשימה למעלה אם אתה רוצה להעיף גם בהונות

# --- EXECUTE ---
df_raw = filter_and_verify_joints(df_raw, DROP_KEYWORDS)


==================== JOINT FILTERING & VERIFICATION ====================
📉 Dropped 210 columns matching: ['Thumb', 'Index', 'Middle', 'Ring', 'Pinky']

📊 STATUS REPORT:
   Original Columns: 359
   Current Columns:  149
   Unique Joints Remaining: 21

✅ LIST OF JOINTS KEPT (21):
----------------------------------------
   Head, Hips, LeftArm, LeftFoot
   LeftForeArm, LeftHand, LeftLeg, LeftShoulder
   LeftToeBase, LeftUpLeg, Neck, RightArm
   RightFoot, RightForeArm, RightHand, RightLeg
   RightShoulder, RightToeBase, RightUpLeg, Spine
   Spine1
----------------------------------------



In [13]:
# --- UPDATED CELL 3: Build Map form Remaining Joints ---

def build_map_from_available_joints(df_columns, hierarchy_dict):
    """
    Scans the current DataFrame columns and builds the kinematics map 
    ONLY for the joints that actually exist in the data.
    """
    print(f"\n{'='*20} BUILDING KINEMATICS MAP {'='*20}")
    
    kinematics_map = {}
    
    
    existing_segments = set([c.split('__')[0] for c in df_columns if '__' in c])
    
    skipped_count = 0
    kept_count = 0
    
    for segment, info in hierarchy_dict.items():
        parent = info['parent']
        angle_name = info['angle_name']
        
        
        if segment not in existing_segments:
            
            skipped_count += 1
            continue
            
        if parent is not None and parent not in existing_segments:
            print(f"⚠️ WARNING: Orphaned Joint '{segment}' (Parent '{parent}' is missing). Skipping.")
            continue

        # 3. הוספה למפה
        kinematics_map[segment] = {
            "parent": parent,
            "angle_name": angle_name,
            "is_global": (parent is None)
        }
        kept_count += 1

    print(f"Total defined in Schema: {len(hierarchy_dict)}")
    print(f"Skipped (Filtered out):  {skipped_count}")
    print(f"Mapped (Ready for calc): {kept_count}")
    print(f"{'='*45}\n")
    
    return kinematics_map

# --- EXECUTE ---
kinematics_map = build_map_from_available_joints(df_raw.columns, SKELETON_HIERARCHY)


==================== BUILDING KINEMATICS MAP ====================
Total defined in Schema: 27
Skipped (Filtered out):  6
Mapped (Ready for calc): 21



In [14]:
# Configuration
MAX_GAP_SIZE = 10  # Max frames to interpolate (approx 83ms at 120Hz)

def fill_missing_data(df, max_gap):
    df_clean = df.copy()
    
    # 1. Linear Interpolate (Works well for Pos and small Rot gaps)
    # We interpolate everything first
    df_clean = df_clean.interpolate(method='linear', limit=max_gap, limit_direction='both')
    
    # 2. Normalize Quaternions (Critical after Lerp!)
    # Iterate only over rotation columns to re-normalize
    quat_cols = [c for c in df_clean.columns if c.endswith(('__qx', '__qy', '__qz', '__qw'))]
    segments = set(c.split('__')[0] for c in quat_cols)
    
    for seg in segments:
        qx = df_clean[f"{seg}__qx"]
        qy = df_clean[f"{seg}__qy"]
        qz = df_clean[f"{seg}__qz"]
        qw = df_clean[f"{seg}__qw"]
        
        # Calculate norm
        norms = np.sqrt(qx**2 + qy**2 + qz**2 + qw**2)
        
        # Avoid division by zero (though interpolate handles NaNs)
        norms[norms == 0] = 1.0 
        
        # Normalize
        df_clean[f"{seg}__qx"] /= norms
        df_clean[f"{seg}__qy"] /= norms
        df_clean[f"{seg}__qz"] /= norms
        df_clean[f"{seg}__qw"] /= norms
        
    return df_clean

print("Running Gap Filling...")
df_preprocessed = fill_missing_data(df_raw, MAX_GAP_SIZE)

# Check remaining NaNs (Large gaps)
remaining_nans = df_preprocessed.isna().sum().sum()
print(f"Gap Filling Complete.")
print(f"Remaining NaNs (Large Gaps): {remaining_nans}")

Running Gap Filling...


Gap Filling Complete.
Remaining NaNs (Large Gaps): 0


In [15]:
# --- Cell 5 Replacement: Statistical QC Report ---

def print_missing_data_report(df, run_id):
    total_cells = df.size
    total_nans = df.isna().sum().sum()
    nan_percent = (total_nans / total_cells) * 100
    
    print(f"\n{'='*20} QC REPORT: MISSING DATA {'='*20}")
    print(f"Run ID:       {run_id}")
    print(f"Total Frames: {len(df)}")
    print(f"Total Values: {total_cells}")
    print(f"Total NaNs:   {total_nans}")
    print(f"Cleanliness:  {100 - nan_percent:.4f}% (Target: 100%)")
    
    if total_nans == 0:
        print("\n✅ PERFECT: No missing data detected.")
    else:
        print("\n⚠️  WARNING: Data Gaps Detected!")
        print("-" * 60)
        print(f"{'Column Name':<35} | {'Missing Frames':<15} | {'% Missing':<10}")
        print("-" * 60)
        
        # Find columns with NaNs and sort by severity
        missing_counts = df.isna().sum()
        missing_cols = missing_counts[missing_counts > 0].sort_values(ascending=False)
        
        for col, count in missing_cols.items():
            pct = (count / len(df)) * 100
            print(f"{col:<35} | {count:<15} | {pct:.2f}%")
            
        print("-" * 60)
        print("Note: Gaps larger than the interpolation limit remain as NaNs.")

# Run the report
print_missing_data_report(df_preprocessed, RUN_ID)


==================== QC REPORT: MISSING DATA ====================
Run ID:       734_T1_P2_R1_Take 2025-12-01 02.28.24 PM
Total Frames: 19617
Total Values: 2922933
Total NaNs:   0
Cleanliness:  100.0000% (Target: 100%)

✅ PERFECT: No missing data detected.


In [16]:
# --- QC Stage: Bone Length Check (Fail Fast) ---

def run_bone_length_qc(df, hierarchy, cfg):
    """
    Quality Gate: Checks if limbs stretch/shrink over time.
    Since we assume a Rigid Body, distance between Parent->Child must be constant.
    """
    print(f"\n{'='*20} BONE LENGTH QC {'='*20}")
    
    # Thresholds from config
    thresh_warn = cfg['THRESH'].get('BONE_CV_WARN', 0.02)   # Default 2%
    thresh_alert = cfg['THRESH'].get('BONE_CV_ALERT', 0.05) # Default 5%
    
    results = []
    
    # Calculate for each defined bone in hierarchy
    for child_name, info in hierarchy.items():
        parent_name = info['parent']
        
        # Skip root (no parent)
        if parent_name is None:
            continue
            
        # Get columns from the processed dataframe
        # Note: In NB02 columns are named "Joint__px", "Joint__py", etc.
        try:
            c_pos = df[[f"{child_name}__px", f"{child_name}__py", f"{child_name}__pz"]].values
            p_pos = df[[f"{parent_name}__px", f"{parent_name}__py", f"{parent_name}__pz"]].values
        except KeyError:
            # If joint was excluded or missing
            continue
        
        # Euclidean Distance
        lengths = np.linalg.norm(c_pos - p_pos, axis=1)
        
        # Stats
        mean_l = np.nanmean(lengths)
        std_l = np.nanstd(lengths)
        
        # Coefficient of Variation (CV)
        cv = std_l / mean_l if mean_l > 0 else 0.0
        
        status = "PASS"
        if cv > thresh_alert:
            status = "FAIL 🔴"
        elif cv > thresh_warn:
            status = "WARN 🟡"
            
        results.append({
            "Bone": f"{parent_name}->{child_name}",
            "Mean_mm": round(mean_l, 1),
            "CV%": round(cv * 100, 2),
            "Status": status
        })

    # Report
    df_qc = pd.DataFrame(results).sort_values("CV%", ascending=False)
    
    n_fails = sum(df_qc['Status'].str.contains("FAIL"))
    n_warns = sum(df_qc['Status'].str.contains("WARN"))
    
    print(f"Checked {len(df_qc)} bones.")
    if n_fails > 0:
        print(f"⛔ CRITICAL: {n_fails} bones failed rigidity check!")
        print("Top offenders:")
        display(df_qc.head(51))
        # Optional: Raise error to stop pipeline if strictly required
        # raise ValueError("Data quality too low: Bones are stretching.")
    elif n_warns > 0:
        print(f"⚠️ WARNING: {n_warns} bones show slight instability.")
        display(df_qc.head(51))
    else:
        print("✅ SUCCESS: Rigid body assumption holds perfectly.")
        
    return df_qc

# Execute on the processed dataframe (df_proc) using the map we built earlier (kinematics_map)
df_bone_qc = run_bone_length_qc(df_preprocessed, kinematics_map, CONFIG)


==================== BONE LENGTH QC ====================
Checked 20 bones.
⚠️ WARNING: 1 bones show slight instability.


,Bone,Mean_mm,CV%,Status
0,Hips->Spine,81.4,3.41,WARN 🟡
3,Neck->Head,145.1,1.93,PASS
1,Spine->Spine1,244.2,1.14,PASS
2,Spine1->Neck,256.7,1.10,PASS
12,Spine1->LeftShoulder,198.8,0.00,PASS
18,RightArm->RightForeArm,325.2,0.00,PASS
17,RightShoulder->RightArm,157.9,0.00,PASS
16,Spine1->RightShoulder,198.8,0.00,PASS
15,LeftForeArm->LeftHand,190.8,0.00,PASS
14,LeftArm->LeftForeArm,325.2,0.00,PASS


In [17]:
# 1. Save Data (Parquet)
out_parquet_path = os.path.join(DERIV_02, f"{RUN_ID}__preprocessed.parquet")
df_preprocessed.to_parquet(out_parquet_path, index=False)

# 2. Save Kinematics Map (JSON)
# This map tells the next notebook WHAT angles to compute
out_map_path = os.path.join(DERIV_02, f"{RUN_ID}__kinematics_map.json")
with open(out_map_path, 'w') as f:
    json.dump(kinematics_map, f, indent=4)

print(f"\n✅ SUCCESS!")
print(f"Data saved to: {out_parquet_path}")
print(f"Map saved to:  {out_map_path}")
print("Proceed to Notebook 03 (Resample).")


✅ SUCCESS!
Data saved to: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_02_preprocess\734_T1_P2_R1_Take 2025-12-01 02.28.24 PM__preprocessed.parquet
Map saved to:  c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_02_preprocess\734_T1_P2_R1_Take 2025-12-01 02.28.24 PM__kinematics_map.json
Proceed to Notebook 03 (Resample).


In [18]:
# --- FINAL CELL: Export Preprocessing Summary for Master Report ---

def export_preprocess_summary(df_raw, df_proc, df_bone_qc, run_id, save_dir, cfg):
    # 1. חישוב נתוני פערים (Gaps)
    total_cells = df_raw.size
    total_nans_pre = df_raw.isna().sum().sum()
    total_nans_post = df_proc.isna().sum().sum()
    
    # 2. סיכום Bone QC
    mean_cv = df_bone_qc['CV%'].mean() if not df_bone_qc.empty else 0
    alerts = len(df_bone_qc[df_bone_qc['Status'].str.contains("FAIL|ALERT")])
    
    summary = {
        "run_id": run_id,
        "raw_missing_percent": round((total_nans_pre / total_cells) * 100, 2),
        "post_missing_percent": round((total_nans_post / total_cells) * 100, 2),
        "max_interpolation_gap": cfg.get('MAX_GAP_SIZE', 10),
        "bone_qc_mean_cv": round(mean_cv, 3),
        "bone_qc_alerts": alerts,
        "worst_bone": df_bone_qc.iloc[0]['Bone'] if not df_bone_qc.empty else "None",
        "interpolation_method": "linear" # או מה שהגדרת
    }
    
    # שמירה ל-JSON
    out_path = os.path.join(save_dir, f"{run_id}__preprocess_summary.json")
    with open(out_path, 'w') as f:
        json.dump(summary, f, indent=4)
    
    print(f"✅ Preprocess Summary exported to: {out_path}")

# הפעלה
export_preprocess_summary(df_raw, df_preprocessed, df_bone_qc, RUN_ID, DERIV_02, CONFIG)

✅ Preprocess Summary exported to: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_02_preprocess\734_T1_P2_R1_Take 2025-12-01 02.28.24 PM__preprocess_summary.json
